# DSCI 525 - Web and Cloud Computing

## Team 14
- 	Nishar	Dhruvi	
- 	Ouedraogo	Flora
- 	Hong	Eyre
- 	Kohli	Manvir

## Introduction 

- (finish later)

- Our repo can be found on Github: https://github.com/UBC-MDS/525_group_14


In [1]:
# Importing packages for analysis
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

### 1. Downloading the data 
rubric={correctness:10}

1. Download the data from [figshare](https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681) to your local computer using the [figshare API](https://docs.figshare.com) (you need to make use of `requests` library).

2. Extract the zip file, again programmatically, similar to how we did it in class. 

>  You can download the data and unzip it manually. But we learned about APIs, so we can do it in a reproducible way with the `requests` library, similar to how we [did it in class](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture1.html#using-rest-api-lab-lecture).

> There are 5 files in the figshare repo. The one we want is: `data.zip`


> The running time depends on different operating system. Below is Eyre's example.

In [2]:
# change to the directory that you want to download
%cd /Users/suno/mds/525/figshareexp

/Users/suno/mds/525/figshareexp


In [3]:
# uncomment this code if it is not your first time to download the data
# %rm rainfall/

In [4]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "rainfall/"

Below we are sending a `GET` request to list the available files

In [5]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]             
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

The below chunk is used to download the data.

In [6]:
%%time
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 12.1 s, sys: 28 s, total: 40.1 s
Wall time: 32min 17s


The below chunck is used to unzip the downloaded zipped data.

In [7]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 15.3 s, sys: 3.21 s, total: 18.5 s
Wall time: 20.9 s


In [8]:
%ls -ltr rainfall/

total 12233984
-rw-r--r--   1 suno  staff  814041183 Mar 28 22:02 data.zip
-rw-r--r--   1 suno  staff   95376895 Mar 28 22:02 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 suno  staff   94960113 Mar 28 22:02 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 suno  staff   82474546 Mar 28 22:02 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 suno  staff  127613760 Mar 28 22:02 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 suno  staff  232118894 Mar 28 22:02 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 suno  staff  330360682 Mar 28 22:02 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 suno  staff  254009247 Mar 28 22:02 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 suno  staff  235661418 Mar 28 22:02 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 suno  staff  294260911 Mar 28 22:02 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 suno  staff  295768615 Mar 28 22:02 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 suno  staff  328852379 Mar 28 22:02 CMCC-ES

### 2. Combining data CSVs
rubric={correctness:10,reasoning:10}

1. Combine data CSVs into a single CSV using pandas.
    
2. When combining the CSV files, add an extra column called "model" that identifies the model.
    Tip 1: you can get this column populated from the file name, eg: for file name "SAM0-UNICON_daily_rainfall_NSW.csv", the model name is SAM0-UNICON
    Tip 2: Remember how we added "year" column when we combined airline CSVs. Here the regex will be to get word before an underscore ie, "/([^_]*)"

> Note: There is a file called `observed_daily_rainfall_SYD.csv` in the data folder that you downloaded. Make sure you exclude this file (programmatically or just take out that file from the folder) before you combine CSVs. We will use this file in our next milestone.

3. ***Compare*** run times on different machines within your team and summarize your observations. 

> Warning: Some of you might not be able to do it on your laptop. It's fine if you're unable to do it. Just make sure you discuss the reasons why you might not have been able to run this on your laptop. 


In [9]:
%cd rainfall

/Users/suno/mds/525/figshareexp/rainfall


In [10]:
%%time
rainfall = pd.DataFrame()
for file in os.listdir("."):
    if file[-3:] == 'csv' and file != 'observed_daily_rainfall_SYD.csv' and file != 'daily_rainfall.csv':
        print(file)
        data = pd.read_csv(file)
        data['model'] = file.split("_")[0]
        rainfall = pd.concat([rainfall, data])

MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
NorESM2-LM_daily_rainfall_NSW.csv
ACCESS-CM2_daily_rainfall_NSW.csv
FGOALS-f3-L_daily_rainfall_NSW.csv
CMCC-CM2-HR4_daily_rainfall_NSW.csv
MRI-ESM2-0_daily_rainfall_NSW.csv
GFDL-CM4_daily_rainfall_NSW.csv
BCC-CSM2-MR_daily_rainfall_NSW.csv
EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
CMCC-ESM2_daily_rainfall_NSW.csv
NESM3_daily_rainfall_NSW.csv
MPI-ESM1-2-LR_daily_rainfall_NSW.csv
ACCESS-ESM1-5_daily_rainfall_NSW.csv
FGOALS-g3_daily_rainfall_NSW.csv
INM-CM4-8_daily_rainfall_NSW.csv
MPI-ESM1-2-HR_daily_rainfall_NSW.csv
TaiESM1_daily_rainfall_NSW.csv
NorESM2-MM_daily_rainfall_NSW.csv
CMCC-CM2-SR5_daily_rainfall_NSW.csv
KIOST-ESM_daily_rainfall_NSW.csv
INM-CM5-0_daily_rainfall_NSW.csv
MIROC6_daily_rainfall_NSW.csv
BCC-ESM1_daily_rainfall_NSW.csv
GFDL-ESM4_daily_rainfall_NSW.csv
CanESM5_daily_rainfall_NSW.csv
SAM0-UNICON_daily_rainfall_NSW.csv
CPU times: user 1min 13s, sys: 31.1 s, total: 1min 44s
Wall time: 1min 51s


In [11]:
rainfall.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


In [12]:
rainfall.shape

(62467843, 7)

In [13]:
%%time
rainfall.to_csv("rainfall.csv", index=False)

CPU times: user 4min 31s, sys: 8.91 s, total: 4min 39s
Wall time: 4min 45s


In [14]:
%%sh
cksum rainfall.csv

3505840762 5970596177 rainfall.csv


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken to merge csv | Time taken to write csv |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|:----------:|
| Nishar	Dhruvi    |                  |     |    |       |        |            |
| Ouedraogo	Flora   |                  |     |     |      |        |            |
| Hong	Eyre    | macOS 13.2.1 | 16 GB  | 2 GHz Quad-Core Intel Core i5  |  Yes | 1min 51s  |  4min 45s  |
| Kohli	Manvir    |        |          |     |           |        |            |


### 3. Load the combined CSV to memory and perform a simple EDA
rubric={correctness:10,reasoning:10}

1. Investigate at least two of the following approaches to reduce memory usage while performing the EDA (e.g., value_counts). Refer to lecture notes [here](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture1.html#some-tactics-to-deal-with-memory-issue).
    - Changing `dtype` of your data
    - Load just columns that we want
    - Loading in chunks
    
2. ***Compare*** run times on different machines within your team and summarize your observations.

### 3.1.1  Loading the entire table all at once.

In [15]:
%%time
df = pd.read_csv("rainfall.csv")
print(df.value_counts())

time                 lat_min     lat_max     lon_min   lon_max   rain (mm/day)  model        
1889-01-01           -35.625000  -33.750000  140.6250  142.5000  0.005392       KIOST-ESM        1
1973-01-01 12:00:00  -36.455696  -34.430380  145.0000  147.0000  0.981719       FGOALS-g3        1
1973-01-01           -31.875000  -30.000000  148.1250  150.0000  4.668059       KIOST-ESM        1
                                             150.0000  151.8750  2.757193       KIOST-ESM        1
                                             151.8750  153.7500  3.492911       KIOST-ESM        1
                                                                                                ..
1931-01-01 12:00:00  -34.430380  -32.405063  153.0000  155.0000  0.169899       FGOALS-g3        1
                     -34.375000  -33.125000  141.5625  143.4375  0.000000       ACCESS-ESM1-5    1
                                             143.4375  145.3125  0.000000       ACCESS-ESM1-5    1
               

### 3.1.2  Loading columns we want.

In [16]:
%%time
use_cols = ['model']
df = pd.read_csv("rainfall.csv",usecols=use_cols)
print(df.value_counts())

model           
MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
dtype: int64
CPU times: user 24.5 s, sys: 3.74 s, total: 28.3 s
Wall time: 28.8 s


### 3.1.3 Loading in chunks

In [17]:
%%time
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("rainfall.csv", chunksize=10_000_000):
    counts = counts.add(chunk[chunk.lat_min < -30]["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1610700
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         2759400
FGOALS-g3           1287720
GFDL-CM4            2759400
INM-CM4-8           1287720
INM-CM5-0           1287720
KIOST-ESM            965790
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       4509960
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
CPU times: user 48.7 s, sys: 5.74 s, total: 54.5 s
Wall time: 54.6 s


### 3.2 ***Compare*** run times on different machines within your team and summarize your observations.

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken to load all | Time taken to load columns we want | Time taken to load in chunks |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|:----------:|:----------:|
| Nishar	Dhruvi    |                  |     |    |       |        |            | |
| Ouedraogo	Flora   |                  |     |     |      |        |            | |
| Hong	Eyre    | macOS 13.2.1 | 16 GB  | 2 GHz Quad-Core Intel Core i5  |  Yes | 28.8s  |  54.6 s  | 1min 8s |
| Kohli	Manvir    |        |          |     |           |        |            | |


### 4. Perform a simple EDA in R
rubric={correctness:15,reasoning:10}

1. Choose one of the methods listed below for transferring the dataframe (i.e., the entire dataset) from Python to R, and explain why you opted for this approach instead of the others.
    - [Parquet file](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture2.html#converting-csv-parquet)
    - [Pandas exchange](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture1.html#use-r-and-python-interchangeably)
    - [Arrow exchange](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture2.html#use-r-and-python-interchangeably-with-arrow)
2. Once you have the dataframe in R, perform a simple EDA.